# Clone Repo & Download Model

In [ ]:
!git clone https://github.com/DialecticalHTR/EasyOCR.git

In [ ]:
url = "https://drive.google.com/open?id=1Jk4eGD7crsqCCg9C9VjCLkMN3ze8kutZ" # paste the url of your model
!gdown --fuzzy {url}

# YAML Config

In [ ]:
%%writefile "/kaggle/working/EasyOCR/trainer/craft/config/custom_data_train.yaml"
wandb_opt: True
wandb_opt_entity: claidyadress003-narfu

results_dir: "/kaggle/working/EasyOCR/trainer/craft/exp"
vis_test_dir: "/kaggle/working/EasyOCR/trainer/craft/vis_result/"

data_root_dir: "/kaggle/input/dialectic-cards-craft"
score_gt_dir: None
mode: "weak_supervision" #could also be None


train:
  backbone: vgg  
  freeze: True
  use_synthtext: False # If you want to combine SynthText in train time as CRAFT did, you can turn on this option
  synth_data_dir: "/data/SynthText/"
  synth_ratio: 5
  real_dataset: custom
  ckpt_path: "/kaggle/working/craft_mlt_25k.pth"
  eval_interval: 100
  batch_size: 8
  st_iter: 0
  end_iter: 2500
  lr: 0.0001
  lr_decay: 7500
  gamma: 0.2
  weight_decay: 0.00001
  num_workers: 0 # On single gpu, train.py execution only works when num worker = 0 / On multi-gpu, you can set num_worker > 0 to speed up
  amp: True
  loss: 2
  neg_rto: 0.3
  n_min_neg: 5000
  data:
    vis_opt: False
    pseudo_vis_opt: False
    output_size: 768
    do_not_care_label: ['###', '']
    mean: [0.485, 0.456, 0.406]
    variance: [0.229, 0.224, 0.225]
    enlarge_region : [0.5, 0.5] # x axis, y axis
    enlarge_affinity: [0.5, 0.5]
    gauss_init_size: 200
    gauss_sigma: 40
    watershed:
      version: "skimage"
      sure_fg_th: 0.75
      sure_bg_th: 0.05
    syn_sample: -1
    custom_sample: -1
    syn_aug:
      random_scale:
        range: [1.0, 1.5, 2.0]
        option: False
      random_rotate:
        max_angle: 20
        option: False
      random_crop:
        version: "random_resize_crop_synth"
        option: True
      random_horizontal_flip:
        option: False
      random_colorjitter:
        brightness: 0.2
        contrast: 0.2
        saturation: 0.2
        hue: 0.2
        option: True
    custom_aug:
      random_scale:
        range: [ 1.0, 1.5, 2.0 ]
        option: False
      random_rotate:
        max_angle: 20
        option: True
      random_crop:
        version: "random_resize_crop"
        scale: [0.03, 0.4]
        ratio: [0.75, 1.33]
        rnd_threshold: 1.0
        option: True
      random_horizontal_flip:
        option: True
      random_colorjitter:
        brightness: 0.2
        contrast: 0.2
        saturation: 0.2
        hue: 0.2
        option: True

test:
  trained_model : null
  custom_data:
    test_set_size: 10 #NOTE not used in train.py
    test_data_dir: "/kaggle/input/dialectic-cards-craft/" # TODO change when running full training
    text_threshold: 0.75
    low_text: 0.5
    link_threshold: 0.2
    canvas_size: 2240
    mag_ratio: 1.75
    poly: False
    cuda: True
    vis_opt: False

# Fine-tuning

In [ ]:
%cd /kaggle/working/EasyOCR/trainer/craft

In [ ]:
!pip install -r requirements.txt

In [ ]:
!wandb login --relogin a939c4678bbaa791f11eb7378496b6a5bd3543a9

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python3 train.py --yaml=custom_data_train